<a href="https://colab.research.google.com/github/bhavanachem/AI-Python/blob/main/Image_Classif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycocotools #web input
!pip install ffmpeg

from google.colab import drive
drive.mount('/content/drive')

import os
work_dir = "/content/drive/MyDrive/SKILLIT Courses/AI Level 2/Final Project"
os.chdir(work_dir)

from colab_utils import imshow, videoGrabber    #helps show images on colab screen
import numpy as np
import matplotlib.pyplot as plt
from google.colab import output   #to clear screen

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tqdm.auto import tqdm      #status bar, training model status

#define classes
CLASS_NAME = ['Water Bottle', 'Chair', 'Bag']   #adr 0,1,2
#define image capture function

def capture_images(numImage=50, label = 'Null'):
  vid = videoGrabber(showVideo=True, size = (60,40))  #videograbber imported
  img = []
  y=[]

  for x in tqdm(range(numImage)):
    new_image = np.array(vid(0))  #latest image saved in new_image
    img.append(new_image)
    if label!='Null':
      y.append(label) #saving either rock,paper,scissor
  img = np.array(img)
  y = np.array(y)       #converting to numpy array (easier)
  return img,y

#now you have 100 images each for rock paper scissors
#each pixel has val bet 0 and 255 depending upon its brightness
#convert pixels bet 0 and 1 to match weights in neural network

bottle_images, bottle_label = capture_images(numImage = 100, label=0)
chair_images, chair_label = capture_images(numImage=100, label = 1)
bag_images, bag_label = capture_images(numImage=100, label = 2)

train_images = np.concatenate((bottle_images, chair_images, bag_images))
train_images = train_images/255
train_labels = np.concatenate((bottle_label, chair_label, bag_label))

#300 images may not be enough for the model to learn from, so change features to make more images

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

#creates a data generator obj that transforms images
datagen = ImageDataGenerator(
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

idx=0
new_train_images = []
new_train_labels = []
for image in train_images:
  img = image.reshape((1,)+image.shape) #make image 3d, 1x60x40 bc cant tranform without 3d
  i=0
  for batch in datagen.flow(img, save_prefix='test', save_format='jpeg'):
    plt.figure(i)
    #print(batch[0])
    plot = plt.imshow(batch[0])
    i+=1
    new_train_images.append(batch[0])
    new_train_labels.append(train_labels[idx])
    if i > 10:
      break
  idx+=1

  plt.show()

new_train_images=np.array(new_train_images)
new_train_labels=np.array(new_train_labels)

model.summary()

# need to classify images into R,P,S
#dense layer connected, received input from last

model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(3))

model.summary()

#training the model

model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(new_train_images, new_train_labels,epochs=10)

# Extracting accuracy values from the training output
accuracies = [0.8576, 0.9767, 0.9906, 0.9876, 0.9958, 0.9985, 0.9985, 0.9973, 0.9994, 1.0000]

# Calculating average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average accuracy: {average_accuracy}")

!pip install gTTS
from gtts import gTTS
from IPython.display import Audio


# Function to convert text to speech
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save('output.mp3')
    return 'output.mp3'

#output calculation

test_image, _ = capture_images(1)  #label not defined yet

test_image = test_image/255 #pixel 0-1
prediction = model.predict(test_image)
plt.imshow(test_image[0])
plt.title(CLASS_NAME[np.argmax(prediction[0])])

#speak out

# Text you want to convert to speech
text = CLASS_NAME[np.argmax(prediction[0])]


# Convert text to speech and save as an audio file
audio_file = text_to_speech(text)


# Play the audio file
Audio(audio_file, autoplay=True)

model.save('self-made_model.h5') #saving tensorflow = .h5